In [13]:
import os
import requests
import pandas as pd
from tqdm import tqdm

In [18]:
def get_access_token(client_id, client_secret, refresh_token):
    """
    Function to get a new access token
    """
    oauth_url = 'https://www.strava.com/oauth/token'

    payload = {
    'client_id': client_id, 
    'client_secret': client_secret, 
    'refresh_token': refresh_token, 
    'grant_type': 'refresh_token', 
    'f': 'json',
    }

    r = requests.post(oauth_url, data=payload, verify=False)

    access_token = r.json()['access_token']
    return access_token

In [19]:
client_id, client_secret, refresh_token = os.environ['STRAVA_CLIENT_ID'], os.environ['STRAVA_CLIENT_SECRET'], os.environ['STRAVA_CLIENT_REFRESH_TOKEN']

access_token = get_access_token(client_id, client_secret, refresh_token)

/Users/gmacmillan/mambaforge/envs/findingepics/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.strava.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [20]:
def get_data(access_token, per_page=200, page=1): 
    """
    Get authorized athletes strava data
    """
    activities_url = 'https://www.strava.com/api/v3/athlete/activities'
    headers = {'Authorization': 'Bearer ' + access_token}
    params = {'per_page': per_page, 'page': page}

    data = requests.get(
       activities_url, 
       headers=headers, 
       params=params
    ).json()

    return data

In [21]:
max_number_of_pages = 10
data = list()
for page_number in tqdm(range(1, max_number_of_pages + 1)):
    page_data = get_data(access_token, page=page_number)
    if page_data == []:
        break
    data.append(page_data)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]


In [22]:
data_dictionaries = []
for page in data:
    data_dictionaries.extend(page)

print('Number of activities downloaded: {}'.format(len(data_dictionaries)))

Number of activities downloaded: 20


In [23]:
activities = pd.json_normalize(data_dictionaries)

activities[['name', 'distance', 'average_speed', 'moving_time']] \
.sample(5)

KeyError: "None of [Index(['name', 'distance', 'average_speed', 'moving_time'], dtype='object')] are in the [columns]"

In [24]:
page_data

{'message': 'Authorization Error',
 'errors': [{'resource': 'AccessToken',
   'field': 'activity:read_permission',
   'code': 'missing'}]}